In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import os

tqdm.pandas()


# Load raw data

In [3]:
games = pd.read_csv("../data/nfl-big-data-bowl-2024/games.csv")
plays = pd.read_csv("../data/nfl-big-data-bowl-2024/plays.csv")
players = pd.read_csv("../data/nfl-big-data-bowl-2024/players.csv").drop("displayName",axis=1)
tracking_data = pd.concat([pd.read_csv(os.path.join(f"../data/nfl-big-data-bowl-2024/tracking_week_{i}.csv")) for i in range(1, 10)], axis=0)
tackles = pd.read_csv("../data/nfl-big-data-bowl-2024/tackles.csv")


In [26]:
plays["playDescription"][plays["playDescription"].str.contains("TOUCHDOWN", regex=True)].tolist()


['(4:56) (Shotgun) D.Foreman left tackle for 2 yards, TOUCHDOWN.',
 '(2:09) (Shotgun) J.Hurts right end for 26 yards, TOUCHDOWN.',
 '(7:28) (Shotgun) P.Mahomes pass deep left to Ju.Watson for 41 yards, TOUCHDOWN.',
 '(5:33) (No Huddle, Shotgun) K.Murray pass short middle to M.Brown for 25 yards, TOUCHDOWN.',
 '(5:17) (Shotgun) J.Fields pass short middle to D.Montgomery to CHI 32 for 13 yards (L.Vander Esch). FUMBLES (L.Vander Esch), RECOVERED by DAL-M.Parsons at CHI 36. M.Parsons for 36 yards, TOUCHDOWN.',
 '(5:51) (Shotgun) T.Hill right end for 11 yards, TOUCHDOWN.',
 '(4:43) (Shotgun) M.Mariota pass short left to O.Zaccheaus for 19 yards, TOUCHDOWN. TB-M.Edwards was injured during the play.',
 '(11:19) M.Cannon reported in as eligible.  D.Harris left tackle for 5 yards, TOUCHDOWN.',
 '(8:26) E.Elliott right tackle for 14 yards, TOUCHDOWN.',
 '(1:09) (Shotgun) J.Goff pass short left to J.Jackson for 2 yards, TOUCHDOWN.',
 '(6:26) (Shotgun) E.Benjamin up the middle for 5 yards, TOUCHDO

In [10]:
(plays['absoluteYardlineNumber'] + plays['playResult']).describe()


count    12486.000000
mean        67.644642
std         25.572018
min         -8.000000
25%         47.000000
50%         69.000000
75%         88.000000
max        206.000000
dtype: float64

In [51]:
PLAYS_RELEVANT_COLS = ['ballCarrierId', 'ballCarrierDisplayName', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
    'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'absoluteYardlineNumber', 'prePenaltyPlayResult', 'playResult', 'offenseFormation', 'defendersInTheBox', 'passProbability']
non_penalty_non_scoring_plays_raw = plays.loc[(plays['playNullifiedByPenalty'] != "Y") & ~plays["playDescription"].str.contains("TOUCHDOWN", regex=True), PLAYS_RELEVANT_COLS + ['gameId', 'playId']] 

non_penalty_non_scoring_plays_raw = pd.merge(non_penalty_non_scoring_plays_raw, games, how="inner", on="gameId").drop(columns=["homeFinalScore", "visitorFinalScore"]).set_index(["gameId", "playId"])
non_penalty_non_scoring_plays_raw


ballCarrierId ballCarrierDisplayName  quarter  down  \
gameId     playId                                                        
2022100908 3537            48723           Parker Hesse        4     1   
           619             54042          Caleb Huntley        1     1   
           1821            42345         Marcus Mariota        2     1   
           2392            44816      Leonard Fournette        3     1   
           85              44816      Leonard Fournette        1     2   
...                          ...                    ...      ...   ...   
2022110601 1929            46377         Jeffery Wilson        3     1   
           2738            52457         Chase Claypool        3     1   
           242             53440          Justin Fields        1     3   
           1391            47856       David Montgomery        2     1   
           3245            53646         Khalil Herbert        4     2   

                   yardsToGo possessionTeam defensiveTeam yardlineSide  \
gameId     playId                                                        
2022100908 3537           10            ATL            TB          ATL   
           619            10            ATL            TB          ATL   
           1821           10            ATL            TB          ATL   
           2392           10             TB           ATL           TB   
           85             10             TB           ATL           TB   
...                      ...            ...           ...          ...   
2022110601 1929           10            MIA           CHI          MIA   
           2738           10            CHI           MIA          CHI   
           242            10            CHI           MIA          MIA   
           1391           10            CHI           MIA          MIA   
           3245            7            CHI           MIA          CHI   

                   yardlineNumber gameClock  ...  playResult  \
gameId     playId                            ...               
2022100908 3537                41      7:52  ...           9   
           619                 15      4:52  ...          -1   
           1821                32      1:10  ...           8   
           2392                26     10:28  ...           2   
           85                  19     14:53  ...          14   
...                           ...       ...  ...         ...   
2022110601 1929                47     14:24  ...          28   
           2738                35      1:53  ...           4   
           242                 20     11:37  ...           6   
           1391                45      6:03  ...           3   
           3245                18      7:13  ...           5   

                   offenseFormation defendersInTheBox  passProbability  \
gameId     playId                                                        
2022100908 3537             SHOTGUN               7.0         0.747284   
           619               PISTOL               7.0         0.732525   
           1821              PISTOL               7.0         0.789536   
           2392             SHOTGUN               6.0         0.459837   
           85                I_FORM               7.0         0.519393   
...                             ...               ...              ...   
2022110601 1929              I_FORM               8.0         0.375652   
           2738          SINGLEBACK               7.0         0.602379   
           242              SHOTGUN               6.0         0.988321   
           1391          SINGLEBACK               8.0         0.507680   
           3245             SHOTGUN               7.0         0.833829   

                   season  week    gameDate  gameTimeEastern  homeTeamAbbr  \
gameId     playId                                                            
2022100908 3537      2022     5  10/09/2022         13:00:00            TB   
           619       2022     5  10/09/2022         13:00:00   

In [31]:
n_players_per_play = week1.groupby(["gameId", "playId"])["nflId"].nunique()
n_players_per_play.value_counts() # should be 22


NameError: name 'week1' is not defined

In [57]:
non_penalty_plays_raw.loc[(2022090800, 80)]


ballCarrierId                  46076
ballCarrierDisplayName    Josh Allen
quarter                            1
down                               2
yardsToGo                          4
possessionTeam                   BUF
defensiveTeam                     LA
yardlineSide                     BUF
yardlineNumber                    31
gameClock                      14:29
preSnapHomeScore                   0
preSnapVisitorScore                0
passResult                         R
absoluteYardlineNumber            79
prePenaltyPlayResult               7
playResult                         7
offenseFormation               EMPTY
defendersInTheBox                6.0
passProbability             0.699896
season                          2022
week                               1
gameDate                  09/08/2022
gameTimeEastern             20:20:00
homeTeamAbbr                      LA
visitorTeamAbbr                  BUF
Name: (2022090800, 80), dtype: object

In [34]:
test_play = tracking_data.groupby(["gameId", "playId"]).get_group((2022090800, 80))
play_groups = test_play.groupby("nflId")
first_group = play_groups.get_group(list(play_groups.groups.keys())[0])
time_to_first_contact = first_group.event.tolist().index("first_contact")

play_timeseries = pd.concat([g.reset_index(drop=True).loc[:, ["x", "y", "s", "a", "o", "dir"]] for _, g in play_groups], axis=1, keys=play_groups.groups.keys())

truncated_play_timeseries = play_timeseries.loc[:time_to_first_contact]
event_timeseries = first_group.event.reset_index(drop=True).loc[:time_to_first_contact]
player_data_for_play = pd.merge(play_groups.first().loc[:, ["gameId", "playId", "displayName", "time", "club"]], players.drop(columns="collegeName"), how="left", on="nflId")
player_data_for_play_with_tackles = pd.merge(player_data_for_play, tackles, how="left", on=["gameId", "playId", "nflId"])

tackle_label = int(player_data_for_play_with_tackles.loc[:, ["pff_missedTackle"]].sum(axis=0).max() == 0)
play_lookup = non_penalty_plays_raw.loc[*test_play[["gameId", "playId"]].iloc[0]]
direction = 2 * int(first_group["playDirection"].iloc[0] == "right") - 1

ball_carrier_trajectory = play_timeseries.xs(play_lookup.ballCarrierId, axis=1)
yds_at_first_contact = ball_carrier_trajectory.loc[time_to_first_contact, "x"]
yds_final = ball_carrier_trajectory.loc[len(ball_carrier_trajectory) - 1, "x"]
outcome = direction * (yds_final - yds_at_first_contact)
print(tackle_label, outcome)


NameError: name 'non_penalty_plays_raw' is not defined

In [55]:
test_play["nflId"].nunique()


0

In [96]:
RELEVANT_EVENTS = ["first_contact", "ball_snap", "pass_outcome_caught", "handoff", "pass_arrived", "out_of_bounds", "run", "man_in_motion", "play_action", "touchdown", "fumble"]
RELEVANT_GAME_INFO = ["gameId", "playId",  "displayName", "time", "club"]
RELEVANT_PLAYER_PLAY_INFO = ["x", "y", "s", "a", "o", "dir"]

def collect_play_by_play_information(play_data):
    play_groups = play_data.groupby("nflId")
    game_id, play_id = play_data[["gameId", "playId"]].iloc[0]
    first_group = play_groups.get_group(list(play_groups.groups.keys())[0])
    time_to_first_contact = first_group.event.tolist().index("first_contact")

    play_timeseries = pd.concat([g.reset_index(drop=True).loc[:, RELEVANT_PLAYER_PLAY_INFO] for _, g in play_groups], axis=1, keys=play_groups.groups.keys())
    truncated_play_timeseries = play_timeseries.loc[:time_to_first_contact]

    event_timeseries = first_group.event.reset_index(drop=True).loc[:time_to_first_contact]
    event_timeseries_encoded = pd.DataFrame(event_timeseries.values.reshape(-1, 1) == np.array(RELEVANT_EVENTS), columns=RELEVANT_EVENTS).astype(int)
    player_data_for_play = pd.merge(play_groups.first().loc[:, RELEVANT_GAME_INFO], players.drop(columns="collegeName"), how="left", on="nflId")
    player_data_for_play_with_tackles = pd.merge(player_data_for_play, tackles, how="left", on=["gameId", "playId", "nflId"])

    """
        This is the tricky part -- we want the tackle label to correspond to whether an instance of first contact led to a tackle.
        That first contact might not have necessarily led to a tackle, or it may have led to a tackle. We want to make sure that *only*
        contacts leading to a tackle are labeled as positive.

        Therefore, we define the label such that a first contact resulted in a successful tackle if any of the individuals credited for
        the tackle were within 2 yards of the ball carrier. This is a conservative threshold to account for some measurement error in the
        tracking data.
    """

    play_features = non_penalty_non_scoring_plays_raw.loc[(game_id, play_id), PLAYS_RELEVANT_COLS]

    ball_carrier_trajectory = play_timeseries.xs(play_features.ballCarrierId, axis=1)

    carrier_xy = ball_carrier_trajectory.loc[time_to_first_contact, ["x", "y"]]
    
    tacklers = player_data_for_play_with_tackles.loc[(player_data_for_play_with_tackles["tackle"] + player_data_for_play_with_tackles["assist"]) > 0, "nflId"].tolist()
    tackle_label = 0
    for tackler_id in tacklers:
        tackler_xy = play_timeseries.xs(tackler_id, axis=1).loc[time_to_first_contact, ["x", "y"]]
        if np.linalg.norm(tackler_xy - carrier_xy) <= 2:
            tackle_label = 1
    #tackle_label = int(player_data_for_play_with_tackles.loc[:, ["pff_missedTackle"]].sum(axis=0).max() == 0)

    # yards after first_contact
    # play_features_with_game_data = pd.merge(play_features, self.games, how="left", on="gameId")
    direction = 2 * int(first_group["playDirection"].iloc[0] == "right") - 1
    yds_at_first_contact = ball_carrier_trajectory.loc[time_to_first_contact, "x"]
    yds_final = ball_carrier_trajectory.loc[len(ball_carrier_trajectory) - 1, "x"]
    outcome = direction * (yds_final - yds_at_first_contact)

    return {
        "game_id": game_id,
        "play_id": play_id,
        "player_tracking": truncated_play_timeseries,
        "event_timeseries": event_timeseries_encoded,
        "players_on_the_field": player_data_for_play_with_tackles,
        "play_features": play_features,
        #"play_features_encoded": self._play_features.loc[(game_id, play_id)],
        "tackle_successful": tackle_label,
        "yards_after_contact": outcome,
    }

all_play_data_week1 = tracking_data[tracking_data.gameId ==  2022091808].groupby(["gameId", "playId"]).filter(lambda g: ("first_contact" in g["event"].values) and (g.name in non_penalty_non_scoring_plays_raw.index)) \
    .groupby(["gameId", "playId"]).progress_apply(collect_play_by_play_information)


  0%|          | 0/53 [00:00<?, ?it/s]

In [97]:
y = np.array([all_play_data_week1.iloc[i]["tackle_successful"] for i in range(len(all_play_data_week1))])
y.mean()


0.7547169811320755

In [68]:
all_play_data_week1.iloc[i]["player_tracking"]


> c:\users\trent\.conda\envs\nfl_bdb\lib\site-packages\pandas\core\indexes\base.py(3797)get_loc()
   3795             ):
   3796                 raise InvalidIndexError(key)
-> 3797             raise KeyError(key) from err
   3798         except TypeError:
   3799             # If we have a listlike key, _check_indexing_error will raise

> c:\users\trent\.conda\envs\nfl_bdb\lib\site-packages\pandas\core\indexes\multi.py(2865)_get_loc_single_level_index()
   2863             return -1
   2864         else:
-> 2865             return level_index.get_loc(key)
   2866 
   2867     def get_loc(self, key):

> c:\users\trent\.conda\envs\nfl_bdb\lib\site-packages\pandas\core\indexes\multi.py(3276)_get_level_indexer()
   3274 
   3275         else:
-> 3276             idx = self._get_loc_single_level_index(level_index, key)
   3277 
   3278             if level > 0 or self._lexsort_depth == 0:

> c:\users\trent\.conda\envs\nfl_bdb\lib\site-packages\pandas\core\indexes\multi.py(2925)get_loc()
  

In [71]:
tracking_data.loc[(tracking_data.gameId == 2022090800) & (tracking_data.playId == 393)].nflId.unique()


array([34452., 37075., 40107., 40166., 41290., 42395., 42400., 43399.,
       44881., 44976., 46085., 46158., 46180., 46190., 46232., 46376.,
       47853., 47952., 52647., 53459., 53678., 54650.,    nan])